In [1]:
import camelot
import pandas as pd
import sqlite3
from pandasql import sqldf
conn = sqlite3.connect("salas.db")

In [2]:
tables = camelot.read_pdf("salas.pdf",pages='all')
all_dfs = [table.df for table in tables]
combined_df = pd.concat(all_dfs, ignore_index=True)

In [3]:
combined_df.columns = ['Turma','Disciplina','Departamento','Horário','Preferência','Atribuida','Atendida']

In [4]:
indices_para_remover = combined_df[combined_df['Turma'].str.contains("Turma", na=False)].index
combined_df = combined_df.drop(indices_para_remover).reset_index(drop=True)
combined_df = combined_df.drop(columns=['Preferência','Atendida'])

In [5]:
combined_df.to_csv("salas.csv", index=False)
combined_df.to_excel("salas.xlsx", index=False)
combined_df.to_sql("salas", conn, if_exists="replace", index=False)

288

In [9]:
agrupando_por_sala = """
select Atribuida,
GROUP_CONCAT(Horário || ',' || Disciplina,char(10)||char(10)) as disciplina 
FROM combined_df 
GROUP BY (Atribuida) 
ORDER BY Atribuida,Horário
"""
result_df = sqldf(agrupando_por_sala,globals())
result_df.to_csv("salas_agrupadas.csv", index=False)